In [1]:
import os
import csv
import math
from zipfile import ZipFile
# Local clustering class with a bunch of methods
from numpy.ma.core import nonzero
os.chdir("/scratch/nick/MSMC-Curve-Analysis")
# Data thingy libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as snsariTesting

from tslearn.generators import random_walks
import matplotlib.cm as cm
import json 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import kneed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_samples
from sklearn.metrics import adjusted_rand_score
# from sklearn.metrics import silhouette_score
from tslearn.clustering import silhouette_score

from MSMC_clustering import Msmc_clustering
import MSMC_clustering as mc
import Silhouette_analysis as sa

# Load in precomputed list of files to filter out (Saves time)
# Consider making the sharpness filter on percent too
regular_path = "lenient-curve-filter"
filter_path = "MSMC-Exploratory-Analysis/results/figures/filtered-out-curves"
kept_path = "MSMC-Exploratory-Analysis/results/figures/kept-curves"

omit_list_path = "MSMC-Exploratory-Analysis/results/lists"
omit_test_lenient_file = "omit_test_lenient.txt"
kept_test_lenient_file = "kept_test_lenient.txt"

if omit_test_lenient_file not in os.listdir(omit_list_path):
    omit_test_lenient = filter_flattness_sharpness(cluster_rt, 
                        identical_val_threshold = 0.7, err = 0.01, # Args for filtering flattness
                        magnitude_jump_threshold = 0.95, points_per_jump = 3, # Args for filtering magnitude jumps
                        plot = True, min_unique_vals=999, ignore_low_mag=0, 
                        fs_x=10, fs_y=9, save_to_filter=filter_path, save_to_kept=kept_path) # These are some pretty good settings
    with open(omit_list_path+"/"+omit_test_lenient_file, "w") as myFile:
        for fileName in omit_test_lenient:
            myFile.write(f"{fileName}\n")

else: # If list exists
    omit_test_lenient = [] # List of names to files which were filtered out due to being elboq shaped
    with open(omit_list_path+"/"+omit_test_lenient_file, "r") as myFile:
        for line in myFile:
            omit_test_lenient.append(line.rstrip())

kept_test_lenient = [] # List of names to files which were kept
for jpg_name in os.listdir(kept_path):
    jpg_name = jpg_name[:-4] # assuming that all files in "kept_path" are .jpgs and end with the specifier ".jpg"
    kept_test_lenient.append(jpg_name)

# List of latin names is useful for indexing on metadata dfs later on
omit_test_lenient_latin = [name[:name.index("_GC")] for name in omit_test_lenient] 
kept_test_lenient_latin = [name[:name.index("_GC")] for name in kept_test_lenient]

import pickle
from sklearn.metrics import adjusted_rand_score

def writeList(save_to, inlist):
    with open(save_to, "w") as f:
        for x in inlist:
            f.write(x+"\n")
    return
            
def readList(path):
    with open(path, "r") as f:
        output = []
        for x in f:
            x = x.rstrip("\n")
            output.append(x)
    return output

def label2subtable(table, label):
    res = table[table["Labels"]==label]
    return res

def ARI_consistency_test(algo: "str",
                         to_omit: "list<str>",
                         iters: "int",
                         gammas: "list<int>",
                         clusts: "list<int>",
                         oldSeed: "int",
                         newSeeds: "list<int>",
                         save_to: "bool/str"=False)->"dict":
    '''
    Does pairwise comparisons between a reference clustering (using oldSeed)
    and various other clusterings (using newSeeds). Includes option of taking
    in list of sample names (full file name of sample) as items to omit.
    
    Clusterings can be done over a range of gammas (if using softdtw) and over
    a range of manual clustering sizes
    '''
    seeds=newSeeds
    by_gamma_dict = dict() # Contains [old_seed_dict, new_seed_dict, randi_dict]
    for gamma in gammas:
        old_seed_dict = {clust : [] for clust in clusts}
        new_seed_dict = {clust : [] for clust in clusts}
        randi_dict =    {clust : [] for clust in clusts}
        new_clusters = {clust : dict() for clust in clusts}
        old_clusters = {clust : dict() for clust in clusts}
        for clust in clusts:
            cluster_rt_norm_lenient_og = Msmc_clustering(directory="msmc_curve_data/", 
                                                    mu=1.4e-9, 
                                                    generation_time_path='generation_lengths/', 
                                                    real_time=True, 
                                                    to_omit=to_omit,
                                                    normalize_lambda=True, 
                                                    log_scale_time=True, 
                                                    plot_on_log_scale=True, 
                                                    exclude_subdirs=["Archive", "mammals_part_1"], 
                                                    manual_cluster_count=clust,
                                                    algo=algo) # cluster count by sqrt method is 14
            cluster_rt_norm_lenient_og.cluster_curves(omit_front=0, 
                                                      omit_back=0, 
                                                      cols=4,  
                                                      fs_x=60, 
                                                      fs_y=30,
                                                      metric_params={"gamma" : gamma},
                                                      metric="softdtw",
                                                      random_state=oldSeed,
                                                      plot_everything=True,
                                                      iter=iters)
            for seed in seeds:
                cluster_rt_norm_lenient = Msmc_clustering(directory="msmc_curve_data/", 
                                                        mu=1.4e-9, 
                                                        generation_time_path='generation_lengths/', 
                                                        real_time=True, 
                                                        to_omit=to_omit,
                                                        normalize_lambda=True, 
                                                        log_scale_time=True, 
                                                        plot_on_log_scale=True, 
                                                        exclude_subdirs=["Archive", "mammals_part_1"], 
                                                        manual_cluster_count=clust,
                                                        algo=algo) # cluster count by sqrt method is 14

                cluster_rt_norm_lenient.cluster_curves(omit_front=0, 
                                                       omit_back=0, 
                                                       cols=4,  
                                                       fs_x=60, 
                                                       fs_y=30,
                                                       metric_params={"gamma" : gamma},
                                                       metric="softdtw",
                                                       random_state=seed,
                                                       plot_everything=True,
                                                       iter=iters)

                old = cluster_rt_norm_lenient_og.dtw_labels
                old_seed_dict[clust].append(old)

                new = cluster_rt_norm_lenient.dtw_labels
                new_seed_dict[clust].append(new)
                randi_dict[clust].append(adjusted_rand_score(old, new))
                new_clusters[clust][seed] = cluster_rt_norm_lenient
                print(f"[RAND INDEX]: Random seed comparison of {clust} clusters: {adjusted_rand_score(old, new)}")
            old_clusters[clust][oldSeed] = cluster_rt_norm_lenient_og
        by_gamma_dict[gamma] = [old_seed_dict, new_seed_dict, randi_dict, old_clusters, new_clusters]
        if save_to:
            pickle.dump(by_gamma_dict, open(save_to, 'wb'))
    return by_gamma_dict

def badVoteCounter(ARI_consistency_test_dict: "dict",
                   clust: "int",
                   oldSeed: "int",
                   newSeeds: "list<int>",
                   gamma=0.0,
                   savefig_to=False,
                   savepkl_to=False)->"dict":
    '''
    I have seriesDict which is {name: series}

    We have a constant clustering and compare it to N number of random clusterings.

    I will need to separate samples by cluster.
    I add votes to samples for each difference that their constant clustering has
    with each new clustering.

    For each sample in old cluster
        Find clustermates of old sample
        For same sample in new cluster
            Find clustermates of old sample in new cluster
            Count the number of differences between old and new cluster
            Map count of differences to sample name
    '''
    seeds = newSeeds
    old_clusters = ARI_consistency_test_dict[gamma][-2]
    new_clusters = ARI_consistency_test_dict[gamma][-1]
    old = old_clusters[clust][oldSeed]
    oldSampleNames = old.namesofMySeries
    badVoteCount = {sample: 0 for sample in oldSampleNames}
    for oldSampleName in oldSampleNames:
        oldSampleKey = oldSampleName[:oldSampleName.index("_GC")] # Latin name
        oldSampleLabel = old.clusterTable.loc[oldSampleKey]["Labels"]
        oldMates = label2subtable(old.clusterTable, oldSampleLabel) # Clustermates of old sample

        for seed in seeds:
            new = new_clusters[clust][seed]
            newSampleNames = new.namesofMySeries
            newSampleLabel = new.clusterTable.loc[oldSampleKey]["Labels"] # w cluster assignment of old sample
            newMates = label2subtable(new.clusterTable, newSampleLabel)

            difference = list(set(newMates.index) - set(oldMates.index))
            badVoteCount[oldSampleName] += len(difference)

    x, y = zip(*sorted(badVoteCount.items(), key = lambda x:x[1]))
    fig = plt.figure(figsize = (40, 40))
    plt.barh(x, y)
    if savefig_to and savepkl_to:
        plt.savefig(savefig_to)
        pickle.dump(badVoteCount, open(savepkl_to, 'wb'))
    return badVoteCount

def voteOutMovingSample(badVoteCount: "dict",
                        lower_cutoff: "float" = 0.0,
                        cutoff_last: "bool/int" = False,
                        save_to: "str"=False)-> "list":
    # Sort vote dict items by votes (bad samples have many votes)
    x, y = zip(*sorted(badVoteCount.items(), key = lambda x:x[1]))
    y = np.array(y)
    if cutoff_last:
        newOmit = list(x[-cutoff_last:])
    else:
        newOmit = list(x[-len(y[y>lower_cutoff]):]) # Samples with more votes
    newOmit = list(set(newOmit))
    if save_to:
        writeList(save_to, newOmit)
    return newOmit

/home/nick/miniconda3/envs/stats/lib/python3.9/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [18]:
'''
- Get Erik a list of the best performing sample omission list
- Along with that list, send him a table of (samples, seed_iters) = label
- Also get him that time series dataframe
    1.) With all normalized data
    2.) With best performing normalized data

finalARI_test_dict[gamma] is list of len 5

Recall:
by_gamma_dict[gamma] = [old_seed_dict,
                        new_seed_dict,
                        randi_dict,
                        old_clusters,
                        new_clusters]
                        
Silly notes (minor consequences):
- This contains cluster labels of each clustering on seed 999, which is used
  for comparison. Something dumb: I pasted the same thing into the dict for
  each trial :-p
'''

'''
Get into batches_40_trials_cutoff_last_5.100_samples/ and grab each of the:
- ARI_test_dict_oldSeed{XXXX}.pkl
    - Go to "MSMC-Exploratory-Analysis/results/consistency-tests/ARI_testing/"
'''
path = "MSMC-Exploratory-Analysis/results/consistency-tests/ARI_testing/"
thegood = "batches_40_trials_20_cutoff_last_5.100_samples/"
sorted(os.listdir(path+thegood))
final_pickle = "finalARI_test_dict_oldSeed999.pkl"
finalARI_test_dict = pickle.load(open(path+thegood+final_pickle, "rb"))


# READ DATA | Shape filtering and ARI Filtering
seed999clusterlabels = finalARI_test_dict[0.0][0]
finalARI_test_dict[0.0][4][8][944].dtw_labels
oldSeedMC = finalARI_test_dict[0.0][3][8][999]
newSeedsMC = finalARI_test_dict[0.0][4][8]
write_path = 'MSMC-Exploratory-Analysis/results/consistency-tests/'
ts_write_path = 'MSMC-Exploratory-Analysis/results/consistency-tests/ts_data_filtered_for_ARI/'

# MAKE SAMPLE X ITERATION = LABEL TABLE
# Rows will be samples, columns will be iteration/seed id
bestLabelTableName = "best_sample_label_assignments.csv"
newSeedsMC_data_dict = {item[0]: item[1].dtw_labels for item in newSeedsMC.items()}
cols_and_keys = list(newSeedsMC_data_dict.keys())
rows = oldSeedMC.namesofMySeries
cols = cols_and_keys
best_sample_label_df = pd.DataFrame(columns=cols, index=rows)
for key in cols_and_keys:
    best_sample_label_df[key] = newSeedsMC_data_dict[key]
best_sample_label_df.columns = list(range(len(cols_and_keys)))
best_sample_label_df.to_csv(write_path+bestLabelTableName)

# MAKE TIMES X SAMPLE = LAMBDA TABLE
# WARNING: Time series data is unusual, i.e. each point in a time series is 2D
tsDictNew = {item[0]: item[1].drop(["right_time_boundary"], axis=1) for item in oldSeedMC.name2series.items()}
tsDictNew = {item[0]: item[1].rename(columns={"left_time_boundary":"time",
                                              "lambda": "NE"})
             for item in tsDictNew.items()}
tsDictNew
for item in tsDictNew.items():
    fname = item[0] + ".csv"
    df2write = item[1]
    df2write.to_csv(ts_write_path + fname)

In [28]:
path = "MSMC-Exploratory-Analysis/results/consistency-tests/ARI_testing/"
thegood = "batches_1_trials_1_cutoff_last_0.100_samples/"
sorted(os.listdir(path+thegood))
final_pickle = "finalARI_test_dict_oldSeed999.pkl"
finalARI_test_dict = pickle.load(open(path+thegood+final_pickle, "rb"))

In [33]:
len(finalARI_test_dict[0.0][3][8][999])

183

In [35]:
# READ DATA | Same stuff as above but just for control/regular 183 datapoints (Shape filtering only)
seed999clusterlabels = finalARI_test_dict[0.0][0]
# finalARI_test_dict[0.0][4][8][944].dtw_labels
oldSeedMC = finalARI_test_dict[0.0][3][8][999]
newSeedsMC = finalARI_test_dict[0.0][4][8]best_sample_label_df
write_path = 'MSMC-Exploratory-Analysis/results/consistency-tests/'
ts_write_path = 'MSMC-Exploratory-Analysis/results/consistency-tests/control_ts_data_for_ARI/'

# MAKE SAMPLE X ITERATION = LABEL TABLE
# Rows will be samples, columns will be iteration/seed id
bestLabelTableName = "control_sample_label_assignments.csv"
newSeedsMC_data_dict = {item[0]: item[1].dtw_labels for item in newSeedsMC.items()}
cols_and_keys = list(newSeedsMC_data_dict.keys())
rows = oldSeedMC.namesofMySeries
cols = cols_and_keys
best_sample_label_df = pd.DataFrame(columns=cols, index=rows)
for key in cols_and_keys:
    best_sample_label_df[key] = newSeedsMC_data_dict[key]
best_sample_label_df.columns = list(range(len(cols_and_keys)))
best_sample_label_df.to_csv(write_path+bestLabelTableName)

# MAKE TIMES X SAMPLE = LAMBDA TABLE
# WARNING: Time series data is unusual, i.e. each point in a time series is 2D
tsDictNew = {item[0]: item[1].drop(["right_time_boundary"], axis=1) for item in oldSeedMC.name2series.items()}
tsDictNew = {item[0]: item[1].rename(columns={"left_time_boundary":"time",
                                              "lambda": "NE"})
             for item in tsDictNew.items()}
for item in tsDictNew.items():
    fname = item[0] + ".csv"
    df2write = item[1]
    df2write.to_csv(ts_write_path + fname)

In [34]:
# NOW DO STUFF FOR REGULAR, UNFILTERED STUFF
best_sample_label_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
Pteruthius_melanotis_GCA_013401235.1_msmc2.final,0,6,2,0,3,0,5,3,3,1,...,6,4,3,2,3,0,5,3,3,4
Menura_novaehollandiae_GCA_013396355.1_msmc2.final,4,3,1,3,4,1,1,0,2,3,...,0,1,4,6,7,1,0,6,4,6
Atrichornis_clamosus_GCA_013400975.1_msmc2.final,3,5,0,4,0,5,7,7,0,5,...,4,5,0,1,4,3,6,7,6,1
Nothoprocta_ornata_GCA_013398335.1_msmc2.final,7,4,5,3,0,7,3,0,0,2,...,5,2,4,4,1,1,6,4,7,5
Malurus_elegans_GCA_013397995.1_msmc2.final,3,5,0,4,0,5,7,7,0,5,...,4,5,0,1,4,3,6,7,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aleadryas_rufinucha_GCA_013397815.1_msmc2.final,7,4,5,6,0,6,3,1,5,2,...,5,2,4,4,0,7,4,4,7,5
Hemiprocne_comata_GCA_013396855.1_msmc2.final,3,5,1,4,0,7,7,4,0,5,...,4,5,0,1,4,3,6,7,6,5
Herpetotheres_cachinnans_GCA_013399355.1_msmc2.final,4,3,1,3,4,1,1,4,2,3,...,0,1,4,6,1,1,0,6,4,5
Thinocorus_orbignyianus_GCA_013400375.1_msmc2.final,6,7,7,6,5,3,4,3,4,7,...,1,0,6,2,0,7,1,3,1,4
